# **Morelet wavelet FWHM definition**

## This code accompanies the paper:
###    "A better way to define and describe Morlet wavelets for time-frequency analysis"
####     by MX Cohen (NeuroImage, 2019)


Questions? -> mikexcohen@gmail.com

In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec # for subplots
import scipy.io as sio # for importing matlab .mat data

# **Figure 1: The happy time-frequency man**

In [ ]:
### generate signal

# parameters
srate = 1000
time  = np.arange(0,3*srate)/srate
pnts  = len(time)

# frequency ranges
f1 = [3, 20]
f2 = [20, 3]

# transient gaussian
transgaus = np.exp( -(time-np.mean(time))**2 / .2 )

# the signal
signal = np.sin(2*np.pi*time * np.linspace(f1[0],np.mean(f1),pnts)) + \
         np.sin(2*np.pi*time * np.linspace(f2[0],np.mean(f2),pnts)) + \
         np.sin(2*np.pi*time*20) * transgaus

In [ ]:
### static power spectrum

hz = np.linspace(0,srate/2,int(pnts/2)+1)
powr = (2*abs(np.fft.fft(signal)/pnts))**2

In [ ]:
### time-frequency analysis

# tf params
nfrex = 40
frex = np.linspace(2,25,nfrex)
fwhm = np.linspace(.8,.7,nfrex)

# setup wavelet and convolution parameters
wavet = np.arange(-2,2,1/srate)
halfw = int(len(wavet)/2)
nConv = pnts + len(wavet) - 1

# initialize time-frequency matrix
tf = np.zeros((len(frex),pnts))

# spectrum of data
dataX = np.fft.fft(signal,nConv)

# loop over frequencies
for fi,f in enumerate(frex):
    
    # create wavelet
    waveX = np.fft.fft( np.exp(2*1j*np.pi*f*wavet) * np.exp(-4*np.log(2)*wavet**2/fwhm[fi]**2),nConv )
    waveX = waveX / np.abs(max(waveX)) # normalize
    
    # convolve
    ast = np.fft.ifft( waveX*dataX )
    # trim and reshape
    ast = ast[halfw-1:-halfw]
    
    # power
    tf[fi,:] = np.abs(ast)**2


In [ ]:
### plotting

fig = plt.figure(figsize=(8,7),tight_layout=True)
gs = gridspec.GridSpec(2,2)

ax = fig.add_subplot(gs[0,:])
ax.plot(time,signal,'k',linewidth=2)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.set_title('A) Time domain')


ax = fig.add_subplot(gs[1,0])
ax.plot(hz,powr[:len(hz)],'k',linewidth=2)
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Power')
ax.set_xlim([0,30])
ax.set_title('B) Frequency domain')


ax = fig.add_subplot(gs[1,1])
ax.imshow(tf,origin='top',aspect='auto',extent=[time[0],time[-1],frex[0],frex[-1]])
ax.set_xlabel('Time (sec.)')
ax.set_ylabel('Frequency (Hz)')
ax.set_title('C) Time-frequency domain')

plt.show()

# **Figure 2: Big confusion from little wavelets**

In [ ]:
### Create the three wavelets

# simulation parameters
freq1 = 7 # Hz
freq2 = 12
srate = 1000
time  = np.arange(-2000,2001)/srate
pnts  = len(time)

# define number of cycles
numcycles = [ 3, 8 ]

# create the sine waves and Gaussian windows
sinwave1 = np.cos(2*np.pi*freq1*time)
sinwave2 = np.cos(2*np.pi*freq2*time)
gauswin1 = np.exp( -time**2 / (2* (numcycles[0]/(2*np.pi*freq1))**2 ) )
gauswin2 = np.exp( -time**2 / (2* (numcycles[1]/(2*np.pi*freq1))**2 ) )
gauswin3 = np.exp( -time**2 / (2* (numcycles[1]/(2*np.pi*freq2))**2 ) )


# create the three wavelets
morletwave1 = sinwave1 * gauswin1
morletwave2 = sinwave1 * gauswin2
morletwave3 = sinwave2 * gauswin3

In [ ]:
### normalized power spectra of the wavelets

powspect1 = np.abs(np.fft.fft(morletwave1)/pnts)**2
powspect1 = powspect1 / np.max(np.abs(powspect1))

powspect2 = np.abs(np.fft.fft(morletwave2)/pnts)**2
powspect2 = powspect2 / np.max(np.abs(powspect2))

powspect3 = np.abs(np.fft.fft(morletwave3)/pnts)**2
powspect3 = powspect3 / np.max(np.abs(powspect3))

# vector of frequencies
hz = np.linspace(0,srate/2,int(pnts/2)+1)

In [ ]:
### compute the empirical temporal FWHM in seconds (later converted to ms)

# midpoint
midp = np.argmin(np.abs(time))

fwhmT = [0,0,0]
fwhmT[0] = time[ midp+np.argmin(np.abs(gauswin1[midp:]-.5)) ] - time[ np.argmin(np.abs(gauswin1[:midp]-.5)) ]
fwhmT[1] = time[ midp+np.argmin(np.abs(gauswin2[midp:]-.5)) ] - time[ np.argmin(np.abs(gauswin2[:midp]-.5)) ]
fwhmT[2] = time[ midp+np.argmin(np.abs(gauswin3[midp:]-.5)) ] - time[ np.argmin(np.abs(gauswin3[:midp]-.5)) ]

idx1 = np.argmin(np.abs(hz-freq1))
idx2 = np.argmin(np.abs(hz-freq2))
fwhmF = [0,0,0]
fwhmF[0] = hz[idx1+np.argmin(np.abs(powspect1[idx1:]-.5))] - hz[np.argmin(np.abs(powspect1[:idx1]-.5))]
fwhmF[1] = hz[idx1+np.argmin(np.abs(powspect2[idx1:idx1+100]-.5))] - hz[np.argmin(np.abs(powspect2[:idx1]-.5))]
fwhmF[2] = hz[idx2+np.argmin(np.abs(powspect3[idx2:]-.5))] - hz[np.argmin(np.abs(powspect3[:idx2]-.5))]

In [ ]:
### plotting

fig,ax = plt.subplots(2,3,figsize=(10,7),tight_layout=True)

# time domain
ax[0,0].plot(time,morletwave1,'k')
ax[0,0].set_xlabel('Time (s)')
ax[0,0].set_title('%g cycles, FWHM: %g ms' %(numcycles[0],fwhmT[0]*1000))
ax[0,0].set_xlim([-1,1])

ax[0,1].plot(time,morletwave2,'k')
ax[0,1].set_xlabel('Time (s)')
ax[0,1].set_title('%g cycles, FWHM: %g ms' %(numcycles[1],fwhmT[1]*1000))
ax[0,1].set_xlim([-1,1])

ax[0,2].plot(time,morletwave3,'k')
ax[0,2].set_xlabel('Time (s)')
ax[0,2].set_title('%g cycles, FWHM: %g ms' %(numcycles[1],fwhmT[2]*1000))
ax[0,2].set_xlim([-1,1])



# frequency domain
ax[1,0].plot(hz,powspect1[:len(hz)],'k')
ax[1,0].set_xlabel('Frequency (Hz)')
ax[1,0].set_title('%g cycles, FWHM: %g Hz' %(numcycles[0],fwhmF[0]))
ax[1,0].set_xlim([0,freq2*2])

ax[1,1].plot(hz,powspect2[:len(hz)],'k')
ax[1,1].set_xlabel('Frequency (Hz)')
ax[1,1].set_title('%g cycles, FWHM: %g Hz' %(numcycles[1],fwhmF[1]))
ax[1,1].set_xlim([0,freq2*2])

ax[1,2].plot(hz,powspect3[:len(hz)],'k')
ax[1,2].set_xlabel('Frequency (Hz)')
ax[1,2].set_title('%g cycles, FWHM: %g Hz' %(numcycles[1],fwhmF[2]))
ax[1,2].set_xlim([0,freq2*2])

plt.show()

## **Figure 3: FWHM vs. number-of-cycles**

In [ ]:
# specify frequencies
frex = np.linspace(3,60,50)
fwhm = np.linspace(.4,.1,len(frex)) # in ms (use this or the following)
fwhm = np.logspace(np.log10(.4),np.log10(.1),len(frex)) # in ms (use this or the previous)
ncyc = np.linspace(3.2,16,len(frex))



# parameters for complex Morlet wavelets
srate = 1024
wavtime = np.arange(-srate*2,srate*2+1)/srate
midp = np.argmin(np.abs(wavtime))


# outputs
empfwhm = np.zeros((len(frex),2))

# loop over frequencies
for fi in range(len(frex)):
    
    ### create the Gaussian using the FWHM formula (equation 3)
    gwin = np.exp( (-4*np.log(2)*wavtime**2) / fwhm[fi]**2 )
    
    # measure the empirical fwhm
    empfwhm[fi,0] = wavtime[ midp+np.argmin(np.abs(gwin[midp:]-.5)) ] \
                    - wavtime[ np.argmin(np.abs(gwin[:midp]-.5)) ]
    

    ### create the Gaussian using the n-cycles formula (equations 1-2)
    s    = ncyc[fi] / (2*np.pi*frex[fi])
    gwin = np.exp( -wavtime**2 / (2*s**2) )
    
    # empirical FWHM
    empfwhm[fi,1] = wavtime[ midp+np.argmin(np.abs(gwin[midp:]-.5)) ] \
                    - wavtime[ np.argmin(np.abs(gwin[:midp]-.5)) ]


fig = plt.figure(figsize=(8,5))

plt.plot(frex,empfwhm*1000,'o-',markersize=8,markerfacecolor='w')
plt.xlabel('Wavelet frequency (Hz)')
plt.ylabel('FWHM (ms)')
plt.legend(['Using FWHM','Using n-cycles'])
plt.show()

## **Figure 4: Defining wavelets in the frequency domain**

In [ ]:
# specify wavelet parameters
peakf = 11
fwhm  = 5.2 # integers are boring

# specify simulation details
npnts = 8001
srate = 1000


# vector of frequencies
hz = np.linspace(0,srate,npnts)

# frequency-domain Gaussian
s  = fwhm*(2*np.pi-1)/(4*np.pi) # normalized width
x  = hz-peakf;                  # shifted frequencies
fx = np.exp(-.5*(x/s)**2)       # gaussian

# empirical FWHM
idx = np.argmin(np.abs(hz-peakf))
empfwhmF = hz[idx+np.argmin(np.abs(fx[idx:]-.5))] - hz[np.argmin(np.abs(fx[:idx]-.5))]


# time-domain wavelet
morletwavelet = np.fft.fftshift( np.fft.ifft(fx) )
time = np.arange(-np.floor(npnts/2),np.floor(npnts/2)+1)/srate

# FWHM of wavelet in the time domain
midp = np.argmin(np.abs(time))
mw_amp = np.abs(morletwavelet)
mw_amp = mw_amp / np.max(mw_amp)
empfwhmT = time[ midp+np.argmin(np.abs(mw_amp[midp:]-.5)) ] - time[ np.argmin(np.abs(mw_amp[:midp]-.5)) ]

In [ ]:
### plotting

fig,ax = plt.subplots(2,1,figsize=(6,6),tight_layout=True)

# frequency domain
ax[0].plot(hz,fx,'k')
ax[0].set_xlim([0,peakf*3])
ax[0].set_xlabel('Frequency (Hz)')
ax[0].set_ylabel('Amplitude (gain)')
ax[0].set_title('FWHM specified: %g, obtained: %g Hz' %(fwhm,empfwhmF))


# time domain
ax[1].plot(time,np.real(morletwavelet),linewidth=2)
ax[1].plot(time,np.imag(morletwavelet),'--',linewidth=2)
ax[1].plot(time,np.abs(morletwavelet),linewidth=2,color=[.8,.8,.8])
ax[1].set_xlim([-1,1])
ax[1].legend(['Real part','Imag part','Envelope'])
ax[1].set_xlabel('Time (sec.)')
ax[1].set_title('FWHM: %g ms' %(empfwhmT*1000))

plt.show()

## **Figure 5a: The real deal (fixed FWHM)**

In [ ]:
# note: this is one channel from one dataset from 
# Cohen, M.X., 2015. Comparison of different spatial transformations applied to EEG data: A case study of error processing. Int. J. Psychophysiol. 97, 245â€“257

matdat = sio.loadmat('MorletWaveletDefinition_data.mat')

# extract data from EEG structure
EEGtimes  = matdat['EEG'][0][0][0][0]
EEGsrate  = int(matdat['EEG'][0][0][1][0][0])
EEGpnts   = int(matdat['EEG'][0][0][2][0][0])
EEGtrials = int(matdat['EEG'][0][0][3][0][0])
EEGdata   = matdat['EEG'][0][0][4]

In [ ]:
### setup time-frequency analysis

# parameters
nfrex = 80
frex  = np.linspace(2,40,nfrex)
fwhm  = [.1, .5, 2]

# timimg parameters
bidx = [ np.argmin(np.abs(EEGtimes--500)), np.argmin(np.abs(EEGtimes--200)) ]
tidx = np.arange( np.argmin(np.abs(EEGtimes--500)), np.argmin(np.abs(EEGtimes-1300))+1 )

# setup wavelet and convolution parameters
wavet = np.arange(-5*EEGsrate,5*EEGsrate+1)/EEGsrate
halfw = int(len(wavet)/2)+1
nConv = EEGpnts*EEGtrials + len(wavet) - 1


# initialize time-frequency matrix
tf = np.zeros((len(frex),len(tidx),len(fwhm)+1))
empfwhm = np.zeros((len(fwhm)+1,len(frex)))


In [ ]:
### run TF analysis

# setup plot
fig,ax = plt.subplots(1,3,figsize=(10,4))


# spectrum of data
dataX = np.fft.fft(np.reshape(EEGdata,(1,-1),order='F'),nConv)


# loop over FWHM parameter settings
for fwhmi in range(len(fwhm)):
    
    # loop over frequencies
    for fi in range(len(frex)):
        
        # create wavelet
        waveX = np.fft.fft( np.exp(2*1j*np.pi*frex[fi]*wavet) \
                            * np.exp(-4*np.log(2)*wavet**2/fwhm[fwhmi]**2),nConv )
        waveX = waveX / np.max(np.abs(waveX)) # normalize
        
        # convolve
        ast = np.fft.ifft( waveX*dataX )[0]
        # trim and reshape
        ast = np.reshape(ast[halfw-1:-halfw+1],(EEGpnts,EEGtrials),order='F')
        
        # power
        p = np.mean( np.abs(ast)**2 ,axis=1)
        tf[fi,:,fwhmi] = 10*np.log10( p[tidx]/np.mean(p[bidx[0]:bidx[1]]) )
        
        # empirical FWHM
        hz = np.linspace(0,EEGsrate,nConv)
        idx = np.argmin(np.abs(hz-frex[fi]))
        fx = np.abs(waveX)
        empfwhm[fwhmi,fi] = hz[idx+np.argmin(np.abs(fx[idx:]-.5))] - hz[np.argmin(np.abs(fx[:idx]-.5))]

    # plots
    ax[fwhmi].imshow(np.squeeze(tf[:,:,fwhmi]),aspect='auto',\
                      extent=[EEGtimes[tidx[0]],EEGtimes[tidx[-1]],frex[0],frex[-1] ],origin='bottom',\
                      vmin=-2,vmax=2)
    ax[fwhmi].set_title('%g sec' %fwhm[fwhmi])
  

ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('Frequency (Hz)')
plt.show()

In [ ]:
### Figure 5a: The real deal (variable FWHM)

# range of FWHMs
fwhm = np.linspace(1,.2,len(frex))

# loop over frequencies
for fi in range(len(frex)):
    
    # create wavelet
    waveX = np.fft.fft( np.exp(2*1j*np.pi*frex[fi]*wavet) \
                        * np.exp(-4*np.log(2)*wavet**2/fwhm[fi]**2),nConv )
    waveX = waveX / np.max(np.abs(waveX)) # normalize
    
    # convolve
    ast = np.fft.ifft( waveX*dataX )[0]
    # trim and reshape
    ast = np.reshape(ast[halfw-1:-halfw+1],(EEGpnts,EEGtrials),order='F')
    
    # power
    p = np.mean( np.abs(ast)**2 ,axis=1)
    tf[fi,:,3] = 10*np.log10( p[tidx]/np.mean(p[bidx[0]:bidx[1]]) )
    
    # empirical FWHM
    hz = np.linspace(0,EEGsrate,nConv)
    idx = np.argmin(np.abs(hz-frex[fi]))
    fx = np.abs(waveX)
    empfwhm[3,fi] = hz[idx+np.argmin(np.abs(fx[idx:]-.5))] - hz[np.argmin(np.abs(fx[:idx]-.5))]


# plot
plt.imshow(np.squeeze(tf[:,:,3]),aspect='auto',\
                  extent=[EEGtimes[tidx[0]],EEGtimes[tidx[-1]],frex[0],frex[-1] ],origin='bottom',\
                  vmin=-2,vmax=2)
plt.title('%g-%g s' %(fwhm[0],fwhm[-1]))
plt.xlabel('Time (ms)')
plt.ylabel('Frequency (Hz)')
plt.show()

In [ ]:
### Figure 5b: the real deal, part deux

empfwhm[empfwhm>100] = np.nan

fig = plt.figure(figsize=(6,6))

plt.plot(frex,empfwhm.T,'s',markerfacecolor='w',linewidth=2,markersize=8)
plt.legend(['100','500','2000','var'])
plt.ylim([0,18])
plt.xlim([frex[0]-1, frex[-1]+1])
plt.xlabel('Peak frequency (Hz)')
plt.ylabel('FWHM (Hz)')
plt.show()